<p style="text-align:center">
    <a href="https://skills.network/?utm_medium=Exinfluencer&utm_source=Exinfluencer&utm_content=000026UJ&utm_term=10006555&utm_id=NA-SkillsNetwork-Channel-SkillsNetworkCoursesIBMDeveloperSkillsNetworkML311Coursera747-2022-01-01" target="_blank">
    <img src="https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/assets/logos/SN_web_lightmode.png" width="200" alt="Skills Network Logo"  />
    </a>
</p>


# Machine Learning Foundation

## Course 5, Part g: RNN DEMO


## Using RNNs to classify sentiment on IMDB data
For this exercise, we will train a "vanilla" RNN to predict the sentiment on IMDB reviews.  Our data consists of 25000 training sequences and 25000 test sequences.  The outcome is binary (positive/negative) and both outcomes are equally represented in both the training and the test set.

Keras provides a convenient interface to load the data and immediately encode the words into integers (based on the most common words).  This will save us a lot of the drudgery that is usually involved when working with raw text.

We will walk through the preparation of the data and the building of an RNN model.  Then it will be your turn to build your own models (and prepare the data how you see fit).


In [7]:
from tensorflow import keras
from keras.utils import pad_sequences
from keras.models import Sequential
from keras.layers import Dense, Embedding
from keras.layers import SimpleRNN
from keras.datasets import imdb
from keras import initializers

In [8]:
max_features = 20000  # This is used in loading the data, picks the most common (max_features) words
maxlen = 30  # maximum length of a sequence - truncate after this
batch_size = 32

In [9]:
## Load in the data.  The function automatically tokenizes the text into distinct integers
(x_train, y_train), (x_test, y_test) = imdb.load_data(num_words=max_features)
print(len(x_train), 'train sequences')
print(len(x_test), 'test sequences')

25000 train sequences
25000 test sequences


In [10]:
# This pads (or truncates) the sequences so that they are of the maximum length
x_train = pad_sequences(x_train, maxlen=maxlen)
x_test = pad_sequences(x_test, maxlen=maxlen)
print('x_train shape:', x_train.shape)
print('x_test shape:', x_test.shape)

x_train shape: (25000, 30)
x_test shape: (25000, 30)


In [11]:
x_train[123,:]  #Here's what an example sequence looks like

array([  219,   141,    35,   221,   956,    54,    13,    16,    11,
        2714,    61,   322,   423,    12,    38,    76,    59,  1803,
          72,     8, 10508,    23,     5,   967,    12,    38,    85,
          62,   358,    99])

## Keras layers for (Vanilla) RNNs

In this exercise, we will not use pre-trained word vectors.  Rather we will learn an embedding as part of the Neural Network.  This is represented by the Embedding Layer below.

### Embedding Layer
`keras.layers.embeddings.Embedding(input_dim, output_dim, embeddings_initializer='uniform', embeddings_regularizer=None, activity_regularizer=None, embeddings_constraint=None, mask_zero=False, input_length=None)`

- This layer maps each integer into a distinct (dense) word vector of length `output_dim`.
- Can think of this as learning a word vector embedding "on the fly" rather than using an existing mapping (like GloVe)
- The `input_dim` should be the size of the vocabulary.
- The `input_length` specifies the length of the sequences that the network expects.

### SimpleRNN Layer
`keras.layers.recurrent.SimpleRNN(units, activation='tanh', use_bias=True, kernel_initializer='glorot_uniform', recurrent_initializer='orthogonal', bias_initializer='zeros', kernel_regularizer=None, recurrent_regularizer=None, bias_regularizer=None, activity_regularizer=None, kernel_constraint=None, recurrent_constraint=None, bias_constraint=None, dropout=0.0, recurrent_dropout=0.0)`

- This is the basic RNN, where the output is also fed back as the "hidden state" to the next iteration.
- The parameter `units` gives the dimensionality of the output (and therefore the hidden state).  Note that typically there will be another layer after the RNN mapping the (RNN) output to the network output.  So we should think of this value as the desired dimensionality of the hidden state and not necessarily the desired output of the network.
- Recall that there are two sets of weights, one for the "recurrent" phase and the other for the "kernel" phase.  These can be configured separately in terms of their initialization, regularization, etc.






In [12]:
## Let's build a RNN

rnn_hidden_dim = 5
word_embedding_dim = 50
model_rnn = Sequential()
model_rnn.add(Embedding(max_features, word_embedding_dim))  #This layer takes each integer in the sequence and embeds it in a 50-dimensional vector
model_rnn.add(SimpleRNN(rnn_hidden_dim,
                    kernel_initializer=initializers.RandomNormal(stddev=0.001),
                    recurrent_initializer=initializers.Identity(gain=1.0),
                    activation='relu',
                    input_shape=x_train.shape[1:]))

model_rnn.add(Dense(1, activation='sigmoid'))

In [13]:
## Note that most of the parameters come from the embedding layer
model_rnn.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding (Embedding)       (None, None, 50)          1000000   
                                                                 
 simple_rnn (SimpleRNN)      (None, 5)                 280       
                                                                 
 dense (Dense)               (None, 1)                 6         
                                                                 
Total params: 1000286 (3.82 MB)
Trainable params: 1000286 (3.82 MB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________


In [14]:
rmsprop = keras.optimizers.RMSprop(learning_rate = .0001)

model_rnn.compile(loss='binary_crossentropy',
              optimizer=rmsprop,
              metrics=['accuracy'])

In [15]:
model_rnn.fit(x_train, y_train,
          batch_size=batch_size,
          epochs=10,
          validation_data=(x_test, y_test))

Epoch 1/10
782/782 [==============================] - 29s 33ms/step - loss: 0.6786 - accuracy: 0.5808 - val_loss: 0.6442 - val_accuracy: 0.6574
Epoch 2/10
782/782 [==============================] - 20s 26ms/step - loss: 0.5977 - accuracy: 0.6942 - val_loss: 0.5702 - val_accuracy: 0.7032
Epoch 3/10
782/782 [==============================] - 17s 21ms/step - loss: 0.5269 - accuracy: 0.7423 - val_loss: 0.5311 - val_accuracy: 0.7297
Epoch 4/10
782/782 [==============================] - 12s 15ms/step - loss: 0.4808 - accuracy: 0.7720 - val_loss: 0.5058 - val_accuracy: 0.7490
Epoch 5/10
782/782 [==============================] - 10s 13ms/step - loss: 0.4477 - accuracy: 0.7920 - val_loss: 0.4871 - val_accuracy: 0.7616
Epoch 6/10
782/782 [==============================] - 10s 13ms/step - loss: 0.4241 - accuracy: 0.8054 - val_loss: 0.4763 - val_accuracy: 0.7688
Epoch 7/10
782/782 [==============================] - 11s 14ms/step - loss: 0.4058 - accuracy: 0.8152 - val_loss: 0.4675 - val_accuracy:

In [16]:
score, acc = model_rnn.evaluate(x_test, y_test,
                            batch_size=batch_size)
print('Test score:', score)
print('Test accuracy:', acc)

782/782 [==============================] - 3s 4ms/step - loss: 0.4553 - accuracy: 0.7839
Test score: 0.4553268253803253
Test accuracy: 0.7839199900627136


## Exercise

In this exercise, we will illustrate:
- Preparing the data to use sequences of length 80 rather than length 30.  Does it improve the performance?
- Trying different values of the "max_features".  Does this  improve the performance?
- Trying smaller and larger sizes of the RNN hidden dimension.  How does it affect the model performance?  How does it affect the run time?


In [17]:
max_features = 20000  # This is used in loading the data, picks the most common (max_features) words
maxlen = 80  # maximum length of a sequence - truncate after this

(x_train, y_train), (x_test, y_test) = imdb.load_data(num_words=max_features)

x_train = pad_sequences(x_train, maxlen=maxlen)
x_test = pad_sequences(x_test, maxlen=maxlen)

In [18]:
rnn_hidden_dim = 5
word_embedding_dim = 50
model_rnn = Sequential()
model_rnn.add(Embedding(max_features, word_embedding_dim))  #This layer takes each integer in the sequence
model_rnn.add(SimpleRNN(rnn_hidden_dim,
                    kernel_initializer=initializers.RandomNormal(stddev=0.001),
                    recurrent_initializer=initializers.Identity(gain=1.0),
                    activation='relu',
                    input_shape=x_train.shape[1:]))

model_rnn.add(Dense(1, activation='sigmoid'))

In [19]:
rmsprop = keras.optimizers.RMSprop(learning_rate = .0001)

model_rnn.compile(loss='binary_crossentropy',
              optimizer=rmsprop,
              metrics=['accuracy'])

In [20]:
model_rnn.fit(x_train, y_train,
          batch_size=batch_size,
          epochs=10,
          validation_data=(x_test, y_test))

Epoch 1/10
782/782 [==============================] - 29s 34ms/step - loss: 0.6244 - accuracy: 0.6414 - val_loss: 0.5635 - val_accuracy: 0.6964
Epoch 2/10
782/782 [==============================] - 26s 33ms/step - loss: 0.4686 - accuracy: 0.7818 - val_loss: 0.4529 - val_accuracy: 0.7898
Epoch 3/10
782/782 [==============================] - 26s 34ms/step - loss: 0.3936 - accuracy: 0.8255 - val_loss: 0.4142 - val_accuracy: 0.8122
Epoch 4/10
782/782 [==============================] - 28s 36ms/step - loss: 0.3509 - accuracy: 0.8488 - val_loss: 0.3989 - val_accuracy: 0.8188
Epoch 5/10
782/782 [==============================] - 27s 35ms/step - loss: 0.3225 - accuracy: 0.8626 - val_loss: 0.3895 - val_accuracy: 0.8238
Epoch 6/10
782/782 [==============================] - 26s 33ms/step - loss: 0.3011 - accuracy: 0.8746 - val_loss: 0.3720 - val_accuracy: 0.8329
Epoch 7/10
782/782 [==============================] - 26s 33ms/step - loss: 0.2840 - accuracy: 0.8834 - val_loss: 0.3735 - val_accuracy:

In [21]:
max_features = 5000  # This is used in loading the data, picks the most common (max_features) words
maxlen = 80  # maximum length of a sequence - truncate after this

(x_train, y_train), (x_test, y_test) = imdb.load_data(num_words=max_features)

x_train = pad_sequences(x_train, maxlen=maxlen)
x_test = pad_sequences(x_test, maxlen=maxlen)

In [22]:
rnn_hidden_dim = 5
word_embedding_dim = 20
model_rnn = Sequential()
model_rnn.add(Embedding(max_features, word_embedding_dim))  #This layer takes each integer in the sequence
model_rnn.add(SimpleRNN(rnn_hidden_dim,
                    kernel_initializer=initializers.RandomNormal(stddev=0.001),
                    recurrent_initializer=initializers.Identity(gain=1.0),
                    activation='relu',
                    input_shape=x_train.shape[1:]))

model_rnn.add(Dense(1, activation='sigmoid'))

In [23]:
rmsprop = keras.optimizers.RMSprop(learning_rate = .0001)

model_rnn.compile(loss='binary_crossentropy',
              optimizer=rmsprop,
              metrics=['accuracy'])

In [24]:
model_rnn.fit(x_train, y_train,
          batch_size=batch_size,
          epochs=10,
          validation_data=(x_test, y_test))

Epoch 1/10
782/782 [==============================] - 23s 27ms/step - loss: 0.6904 - accuracy: 0.5089 - val_loss: 0.6774 - val_accuracy: 0.5288
Epoch 2/10
782/782 [==============================] - 21s 27ms/step - loss: 0.6385 - accuracy: 0.6556 - val_loss: 0.6085 - val_accuracy: 0.7076
Epoch 3/10
782/782 [==============================] - 21s 27ms/step - loss: 0.5842 - accuracy: 0.7521 - val_loss: 0.5420 - val_accuracy: 0.7553
Epoch 4/10
782/782 [==============================] - 22s 28ms/step - loss: 0.4691 - accuracy: 0.7823 - val_loss: 0.4690 - val_accuracy: 0.7733
Epoch 5/10
782/782 [==============================] - 22s 29ms/step - loss: 0.4217 - accuracy: 0.8095 - val_loss: 0.4327 - val_accuracy: 0.8016
Epoch 6/10
782/782 [==============================] - 19s 24ms/step - loss: 0.3926 - accuracy: 0.8257 - val_loss: 0.4443 - val_accuracy: 0.7844
Epoch 7/10
782/782 [==============================] - 21s 27ms/step - loss: 0.3713 - accuracy: 0.8367 - val_loss: 0.4288 - val_accuracy:

In [25]:
# Out of curiosity, run for 10 more epochs
model_rnn.fit(x_train, y_train,
          batch_size=batch_size,
          epochs=10,
          validation_data=(x_test, y_test))

Epoch 1/10
782/782 [==============================] - 19s 24ms/step - loss: 0.3265 - accuracy: 0.8611 - val_loss: 0.3874 - val_accuracy: 0.8282
Epoch 2/10
782/782 [==============================] - 17s 22ms/step - loss: 0.3208 - accuracy: 0.8648 - val_loss: 0.3791 - val_accuracy: 0.8323
Epoch 3/10
782/782 [==============================] - 17s 22ms/step - loss: 0.3146 - accuracy: 0.8671 - val_loss: 0.3808 - val_accuracy: 0.8295
Epoch 4/10
782/782 [==============================] - 18s 23ms/step - loss: 0.3092 - accuracy: 0.8695 - val_loss: 0.3855 - val_accuracy: 0.8254
Epoch 5/10
782/782 [==============================] - 20s 26ms/step - loss: 0.3042 - accuracy: 0.8732 - val_loss: 0.3790 - val_accuracy: 0.8310
Epoch 6/10
782/782 [==============================] - 16s 20ms/step - loss: 0.2995 - accuracy: 0.8741 - val_loss: 0.3707 - val_accuracy: 0.8369
Epoch 7/10
782/782 [==============================] - 16s 20ms/step - loss: 0.2949 - accuracy: 0.8764 - val_loss: 0.3910 - val_accuracy:

---
### Machine Learning Foundation (C) 2020 IBM Corporation
